In [ ]:
# Done in picasso locally

In [ ]:
import re
import os
import pickle 
import numpy as np 
import pandas as pd
import sys
import joblib

from sklearn import metrics
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.dummy import DummyClassifier

import xgboost as xgb
from xgboost import XGBClassifier


from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [ ]:
with open("drive/MyDrive/OrlyPred/Homomer_embeds/results/embeds_Mar_22/train_set.pkl", 'rb') as f:
  overall_train_set = pickle.load(f)

# index reset is important for the stratified splitting and the saving to lists
overall_train_set.reset_index(drop=True, inplace=True)

In [ ]:
overall_train_set_no_embed = overall_train_set[["code", "nsub", "representative"]]
overall_train_set2 = overall_train_set.copy()
list_of_nsubs = list(set(overall_train_set2["nsub"].tolist()))
for nsub in list_of_nsubs:
    num_of_clusts = overall_train_set_no_embed[overall_train_set_no_embed['nsub'] == nsub].groupby("representative").nunique().shape[0]
    if num_of_clusts < 6:
        overall_train_set2 = overall_train_set2[overall_train_set2.nsub != nsub]
overall_train_set = overall_train_set2


In [ ]:
overall_train_set.reset_index(drop=True, inplace=True)
X = overall_train_set["code"]
y = overall_train_set["nsub"]
groups = overall_train_set["representative"]

In [ ]:
cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=1)
train_lst = []
test_lst = []
for train_idxs, test_idxs in cv.split(X, y, groups):
    train_lst.append(X[train_idxs].tolist())
    test_lst.append(X[test_idxs].tolist())
train_idx_df = pd.DataFrame(train_lst).transpose()
train_idx_df.rename(columns={0:"train_0", 1:"train_1", 2:"train_2", 3:"train_3", 4:"train_4"}, inplace=True)
test_idx_df = pd.DataFrame(test_lst).transpose()
test_idx_df.rename(columns={0:"test_0", 1:"test_1", 2:"test_2", 3:"test_3", 4:"test_4"}, inplace=True)

In [ ]:
train_idx_df


In [ ]:
test_idx_df

In [ ]:
merged_train_test = pd.concat([train_idx_df, test_idx_df], axis=1, join="outer")
train_set = overall_train_set[overall_train_set["code"].isin(merged_train_test["train_0"])]
test_set = overall_train_set[overall_train_set["code"].isin(merged_train_test["test_0"])]

In [ ]:
test_set

In [ ]:
X_train = train_set['embeddings'].tolist()
y_train = train_set['nsub']

X_test = test_set['embeddings'].tolist()
y_test = test_set['nsub']

In [ ]:
def train_from_hyp(best_params, X_train, y_train, groups):
    xgb_model = XGBClassifier(objective='multi:softprob')

    y_train = y_train.values.astype(int)
    le = LabelEncoder()
    y_train = le.fit_transform(y_train)

    xgb_model.set_params(**best_params)

    # generate a scorer suitable for this kind of data
    f1_score_weighted = make_scorer(f1_score, average="weighted")
    f1_score_weighted

    xgb_model.fit(X_train, y_train)

    #Save the final model
    pickle.dump(xgb_model, open("drive/MyDrive/OrlyPred/Homomer_embeds/results/embeds_Mar_22/xgb_model_80.pkl", "wb"))
    joblib.dump(xgb_model, "drive/MyDrive/OrlyPred/Homomer_embeds/results/embeds_Mar_22/xgb_model_80.joblib")

In [ ]:
best_params = {'eta': 0.2,
 'max_depth': 6,
 'min_child_weight': 9,
 'n_estimators': 1500,
 'objective': 'multi:softprob',
 'tree_method': 'approx'}

In [ ]:
xgb_model = XGBClassifier(objective='multi:softprob')

# y_train = y_train.values.astype(int)
# le = LabelEncoder()
# y_train = le.fit_transform(y_train)
X_train = np.asarray(X_train)

xgb_model.set_params(**best_params)

# generate a scorer suitable for this kind of data
f1_score_weighted = make_scorer(f1_score, average="weighted")
f1_score_weighted

xgb_model.fit(X_train, y_train)

#Save the final model
pickle.dump(xgb_model, open("drive/MyDrive/OrlyPred/Homomer_embeds/results/embeds_Mar_22/xgb_model_80.pkl", "wb"))
joblib.dump(xgb_model, "drive/MyDrive/OrlyPred/Homomer_embeds/results/embeds_Mar_22/xgb_model_80.joblib")